## Import - PyCaret 기반 전력사용량 예측

In [1]:
import random
import pandas as pd
import numpy as np
import os

# PyCaret 라이브러리 추가 (GPU 지원)
from pycaret.regression import setup, compare_models, predict_model, finalize_model, create_model, tune_model, evaluate_model

import warnings
warnings.filterwarnings(action='ignore')

# GPU 사용을 위한 환경 설정
os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "INFO"  # GPU 상태 확인을 위해 로그 레벨 증가

# GPU 가용성 확인
import torch
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 장치: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
else:
    print("CUDA를 사용할 수 없습니다. CPU 모드로 실행됩니다.") 

CUDA 사용 가능: True
GPU 장치: NVIDIA GeForce RTX 5070 Ti
GPU 메모리: 15.9GB


## Fixed RandomSeed

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Load Data & Pre-Processing

In [3]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

## '건물번호'==1 인 데이터만 추출

In [14]:
train_df = train_df[train_df['건물번호'] == 1]
test_df = test_df[test_df['건물번호'] == 1]

In [32]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train_df['month'] = train_df['일시'].apply(lambda x : int(x[4:6]))
train_df['day'] = train_df['일시'].apply(lambda x : int(x[6:8]))
train_df['time'] = train_df['일시'].apply(lambda x : int(x[9:11]))

In [33]:
train_x = train_df.drop(columns=['num_date_time', '일시'])
train_y = train_df['전력소비량(kWh)']

In [34]:
# 시계열 특성을 학습에 반영하기 위해 test 데이터도 동일하게 처리합니다
test_df['month'] = test_df['일시'].apply(lambda x : int(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : int(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : int(x[9:11]))

In [35]:
test_x = test_df.drop(columns=['num_date_time', '일시'])

In [40]:
train_df[:1]

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),month,day,time
0,1_20240601 00,1,20240601 00,18.3,0.0,2.6,82.0,0.0,0.0,5794.8,6,1,0


## PyCaret Setup 및 모델 비교

In [ ]:
# PyCaret 환경 설정 (GPU 최적화)
print("GPU 가속을 위한 PyCaret 환경 설정 중...")

exp = setup(
    data=train_df,
    target='전력소비량(kWh)',
    session_id=42,
    verbose=False,
    use_gpu=True,  # GPU 사용 활성화
    feature_selection=False,
    remove_multicollinearity=False,
    ignore_features=['num_date_time', '일시', '건물번호', '일조(hr)', '일사(MJ/m2)'],
    train_size=0.8,  # 학습 데이터 비율 증가 (GPU 효율성)
    fold_strategy='kfold',
    fold=5,
    # GPU 최적화를 위한 추가 설정
    numeric_imputation='mean',
    categorical_imputation='mode',
    transformation=True,
    normalize=True,
    pca=False,
    # GPU 메모리 최적화
    # low_variance_threshold=0.1,
    # multicollinearity_threshold=0.95
)

best_models = []
print("✅ PyCaret 환경 설정 완료 - GPU 가속 모드 활성화")

GPU 가속을 위한 PyCaret 환경 설정 중...
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 5070 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped tr

In [50]:
# GPU 가속 모델들을 우선적으로 비교
print("🚀 GPU 최적화 모델 비교 시작...")

# GPU를 잘 활용하는 모델들 우선 선택
gpu_optimized_models = ['xgboost', 'lightgbm', 'catboost', 'rf', 'et']

best_models = compare_models(
    include=gpu_optimized_models,  # GPU 최적화 모델들
    sort='MAE', 
    n_select=5,
    verbose=True,
    # parallel=True,  # 병렬 처리 활성화
    budget_time=20  # 시간 제한 (분)
)

print("✅ GPU 최적화 모델 비교 완료!")
# print(f"선택된 모델들: {[str(model).split('(')[0] for model in best_models]}")


🚀 GPU 최적화 모델 비교 시작...


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,528.4808,454748.3136,673.4973,0.5486,0.1380,0.1074,0.1040
rf,Random Forest Regressor,557.9381,500650.5809,706.5359,0.5036,0.1451,0.1137,0.1400
lightgbm,Light Gradient Boosting Machine,570.3852,524833.9253,724.0872,0.4780,0.1484,0.1161,0.5280
catboost,CatBoost Regressor,573.4930,522950.9409,722.5808,0.4805,0.1486,0.1170,6.5240
xgboost,Extreme Gradient Boosting,581.7917,556443.3062,745.2398,0.4472,0.1527,0.1182,0.2760


✅ GPU 최적화 모델 비교 완료!


In [56]:
# 바로 예측해보기
best_model = best_models[0]
predictions = predict_model(best_model, data=test_df)
predictions.to_csv(f'./submission_1.csv', index=True)
predictions



,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),month,day,time,prediction_label
0,1_20240825 00,1,20240825 00,26.500000,0.0,0.7,80.0,8,25,0,4323.416614
1,1_20240825 01,1,20240825 01,26.100000,0.0,0.0,80.0,8,25,1,4548.215056
2,1_20240825 02,1,20240825 02,25.900000,0.0,0.3,83.0,8,25,2,4129.113254
3,1_20240825 03,1,20240825 03,25.700001,0.0,1.1,83.0,8,25,3,3420.982783
4,1_20240825 04,1,20240825 04,25.500000,0.0,1.0,86.0,8,25,4,3219.792896
...,...,...,...,...,...,...,...,...,...,...,...
163,1_20240831 19,1,20240831 19,29.700001,0.0,2.5,53.0,8,31,19,6603.629060
164,1_20240831 20,1,20240831 20,28.799999,0.0,2.2,56.0,8,31,20,6309.641401
165,1_20240831 21,1,20240831 21,27.900000,0.0,1.4,66.0,8,31,21,5114.200742
166,1_20240831 22,1,20240831 22,27.400000,0.0,0.3,72.0,8,31,22,5172.269019


In [45]:
# GPU 가속 하이퍼파라미터 튜닝
print("⚡ GPU 가속 하이퍼파라미터 튜닝 시작...")

best_model = best_models[0]
print(f"선택된 최고 성능 모델: {str(best_model).split('(')[0]}")

# GPU 최적화된 튜닝 설정
tuned_model = tune_model(
    best_model, 
    optimize='MAPE', 
    n_iter=10,  # 반복 횟수 증가 (GPU로 빠르게 처리)
    search_library='optuna',  # 고급 최적화 라이브러리
    search_algorithm='tpe',  # Tree-structured Parzen Estimator
    early_stopping=True,  # 조기 종료
    early_stopping_max_iters=10,
    verbose=True
)

print("✅ GPU 가속 하이퍼파라미터 튜닝 완료!")
print(f"튜닝된 모델: {str(tuned_model).split('(')[0]}")


⚡ GPU 가속 하이퍼파라미터 튜닝 시작...
선택된 최고 성능 모델: ExtraTreesRegressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,580.4780,523372.1412,723.4446,0.4430,0.1475,0.1173
1,551.1796,486336.3522,697.3782,0.5087,0.1466,0.1162
2,597.5664,600460.4514,774.8938,0.4740,0.1678,0.1273
3,561.8317,494647.9916,703.3122,0.4879,0.1426,0.1127
4,588.8494,507509.1035,712.3967,0.4892,0.1429,0.1182
Mean,575.9810,522465.2080,722.2851,0.4806,0.1495,0.1183
Std,17.1293,40955.7257,27.7385,0.0218,0.0094,0.0049


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
✅ GPU 가속 하이퍼파라미터 튜닝 완료!
튜닝된 모델: ExtraTreesRegressor


In [58]:
# 모델 성능 시각화 및 평가
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'
evaluate_model(tuned_model)





interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [28]:
# 최종 모델 생성 (전체 데이터로 재학습)
final_model = finalize_model(tuned_model)
print("최종 모델 생성 완료")


최종 모델 생성 완료


## PyCaret 기반 예측

In [31]:
test_df

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),month,day,time
0,1_20240825 00,1,20240825 00,26.5,0.0,0.7,80.0,8,25,0
1,1_20240825 01,1,20240825 01,26.1,0.0,0.0,80.0,8,25,1
2,1_20240825 02,1,20240825 02,25.9,0.0,0.3,83.0,8,25,2
3,1_20240825 03,1,20240825 03,25.7,0.0,1.1,83.0,8,25,3
4,1_20240825 04,1,20240825 04,25.5,0.0,1.0,86.0,8,25,4
...,...,...,...,...,...,...,...,...,...,...
163,1_20240831 19,1,20240831 19,29.7,0.0,2.5,53.0,8,31,19
164,1_20240831 20,1,20240831 20,28.8,0.0,2.2,56.0,8,31,20
165,1_20240831 21,1,20240831 21,27.9,0.0,1.4,66.0,8,31,21
166,1_20240831 22,1,20240831 22,27.4,0.0,0.3,72.0,8,31,22


In [30]:
# test 데이터 예측 (PyCaret 자동 전처리 적용)
predictions = predict_model(final_model, data=test_df)
preds = predictions['전력소비량(kWh)']
print(f"예측 완료: {len(preds)}개 샘플")

KeyError: '전력소비량(kWh)'

## 결과 저장 및 제출

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = preds
print("제출 파일 준비 완료")

In [ ]:
submission.to_csv('./baseline_submission.csv', index=False)
print("PyCaret 기반 예측 결과가 baseline_submission.csv에 저장되었습니다!")
print(f"예측값 통계:")
print(f"- 최소값: {preds.min():.2f}")
print(f"- 최대값: {preds.max():.2f}")
print(f"- 평균값: {preds.mean():.2f}")
print(f"- 표준편차: {preds.std():.2f}")

In [ ]:
# GPU 가속 XGBoost 모델 생성 및 최적화
print("🔥 GPU 가속 XGBoost 모델 생성 중...")

# GPU 최적화 XGBoost 모델 생성
xgb_model = create_model(
    'xgboost',
    tree_method='gpu_hist',  # GPU 가속 히스토그램 방법
    gpu_id=0,  # GPU 장치 ID
    max_bin=256,  # GPU 메모리 최적화
    verbosity=1
)
print("✅ GPU 가속 XGBoost 모델 생성 완료")

# GPU 가속 XGBoost 하이퍼파라미터 튜닝
print("⚡ GPU 가속 XGBoost 튜닝 시작...")
tuned_xgb = tune_model(
    xgb_model, 
    optimize='MAE',
    n_iter=200,  # GPU로 빠른 처리 가능하므로 반복 횟수 증가
    search_library='optuna',
    search_algorithm='tpe',
    early_stopping=True,
    early_stopping_max_iters=15,
    verbose=True
)
print("✅ GPU 가속 XGBoost 하이퍼파라미터 튜닝 완료!")


In [ ]:
# 모델 해석 및 Feature Importance (선택사항)
# interpret_model(tuned_xgb)  # 주석 해제 시 모델 해석 결과 표시


In [26]:
# GPU 성능 모니터링 및 비교
print("📊 GPU 성능 모니터링...")

# GPU 메모리 사용량 확인
if torch.cuda.is_available():
    gpu_memory_allocated = torch.cuda.memory_allocated(0) / 1024**3
    gpu_memory_cached = torch.cuda.memory_reserved(0) / 1024**3
    print(f"GPU 메모리 사용량: {gpu_memory_allocated:.2f}GB")
    print(f"GPU 메모리 캐시: {gpu_memory_cached:.2f}GB")
    
    # GPU 활용률 정보
    print(f"GPU 장치명: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 버전: {torch.version.cuda}")
else:
    print("GPU 정보를 가져올 수 없습니다.")


📊 GPU 성능 모니터링...
GPU 메모리 사용량: 0.00GB
GPU 메모리 캐시: 0.00GB
GPU 장치명: NVIDIA GeForce RTX 5070 Ti
CUDA 버전: 12.1


In [25]:
# GPU 최적화를 위한 추가 패키지 설치 확인
print("🔧 GPU 최적화 패키지 확인...")

try:
    import cudf
    print("✅ cuDF (GPU DataFrame) 사용 가능")
except ImportError:
    print("❌ cuDF 미설치 - GPU DataFrame 가속 불가")

try:
    import cuml
    print("✅ cuML (GPU ML) 사용 가능")
except ImportError:
    print("❌ cuML 미설치 - GPU ML 가속 불가")

try:
    import optuna
    print("✅ Optuna 하이퍼파라미터 최적화 사용 가능")
except ImportError:
    print("❌ Optuna 미설치 - 고급 하이퍼파라미터 튜닝 불가")

print("\n💡 GPU 가속을 최대화하려면 다음 패키지 설치를 권장합니다:")
print("   pip install cudf-cu12 cuml-cu12 optuna")


🔧 GPU 최적화 패키지 확인...
❌ cuDF 미설치 - GPU DataFrame 가속 불가
❌ cuML 미설치 - GPU ML 가속 불가
✅ Optuna 하이퍼파라미터 최적화 사용 가능

💡 GPU 가속을 최대화하려면 다음 패키지 설치를 권장합니다:
   pip install cudf-cu12 cuml-cu12 optuna
